# Block 5 Exercise 2: Predicting the fare of taxi rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [1]:
import pandas as pd
import numpy as np
import folium

In [2]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

In [3]:
# a quick look at the data
X.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,...,pickup_borough_manhattan,pickup_borough_others,pickup_borough_queens,pickup_borough_staten_island,dropoff_borough_bronx,dropoff_borough_brooklyn,dropoff_borough_manhattan,dropoff_borough_others,dropoff_borough_queens,dropoff_borough_staten_island
0,-73.844311,40.721319,-73.841610,40.712278,1,15,17,1,6,2009,...,0,0,1,0,0,0,0,0,1,0
1,-74.016048,40.711303,-73.979268,40.782004,1,5,16,2,1,2010,...,1,0,0,0,0,0,1,0,0,0
2,-73.982738,40.761270,-73.991242,40.750562,2,18,0,4,8,2011,...,1,0,0,0,0,0,1,0,0,0
3,-73.987130,40.733143,-73.991567,40.758092,1,21,4,6,4,2012,...,1,0,0,0,0,0,1,0,0,0
4,-73.968095,40.768008,-73.956655,40.783762,1,9,7,2,3,2010,...,1,0,0,0,0,0,1,0,0,0


In [4]:
y.head()

0     4.5
1    16.9
2     5.7
3     7.7
4     5.3
Name: fare_amount, dtype: float64

### E 2.1 Split data into train and test data
* use data from 2009-1014 for training and 2015 for evaluation


In [5]:
print(X['pickup_year'].unique())

[2009 2010 2011 2012 2013 2014 2015]


In [6]:
lower_2015 = (X['pickup_year'] < 2015)
equal_2015 = (X['pickup_year'] == 2015)

X_train = X[lower_2015]
X_test = X[equal_2015]
y_train = y[lower_2015]
y_test = y[equal_2015]

### E 2.2 Train a Random Forest Regression Model
* train model using [RF regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* optimize the model parameters (#trees, depth, vars per split, ...) and evaluate the [MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) on train and test data
* Analyze the model error: are there typical settings where the model fails?

In [7]:
# baseline with 100 trees, no max depth, minimum number of samples required to split = 2
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('Baseline of RandomForestRegressor score is {}'.format(score))
print('MSE: {}, {} trees, no max. depth, {} min_samples_split'.format(mean_squared_error(y_test, y_pred), 100, 2))

Baseline of RandomForestRegressor score is 0.837026506859996
MSE: 21.350708511472934, 100 trees, no max. depth, 2 min_samples_split


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(), RandomForestRegressor())
pipe.fit(X_train, y_train)  # apply scaling on training data
y_pred = pipe.predict(X_test)
pipe.score(X_test, y_test) 

0.8390133102589654

In [9]:
trees, depth, split = 50, 3, 2
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7304563004069703
MSE: 35.312177767282414, 50 trees, max. depth: 3, 2 min_samples_split


In [10]:
trees, depth, split = 50, 5, 2
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7965036474001421
MSE: 26.659496730398025, 50 trees, max. depth: 5, 2 min_samples_split


In [11]:
trees, depth, split = 50, 5, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.796454954903825
MSE: 26.665875800242663, 50 trees, max. depth: 5, 5 min_samples_split


In [12]:
trees, depth, split = 75, 5, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7974060949943187
MSE: 26.54126955640237, 75 trees, max. depth: 5, 5 min_samples_split


In [13]:
trees, depth, split = 10, 5, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.7973934174841386
MSE: 26.542930402097532, 10 trees, max. depth: 5, 5 min_samples_split


In [14]:
trees, depth, split = 10, 10, 5
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.8254420577037329
MSE: 22.868355292160615, 10 trees, max. depth: 10, 5 min_samples_split


In [15]:
trees, depth, split = 10, 10, 10
rfr = RandomForestRegressor(n_estimators=trees, max_depth=depth, min_samples_split=split)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
score = rfr.score(X_test, y_test)
print('RF-Regressor score is {}'.format(score))
print('MSE: {}, {} trees, max. depth: {}, {} min_samples_split'.format(
    mean_squared_error(y_test, y_pred), trees, depth, split))

RF-Regressor score is 0.8270924576867311
MSE: 22.65214093554702, 10 trees, max. depth: 10, 10 min_samples_split


### Analyse Model Error
*The day of the week, time, date in general and the number of passengers are not taken into account for decision making.*

In [16]:
# The higher, the more important the feature. 
importance = list(map(lambda x: ('%.3f' % x),rfr.feature_importances_))
print(importance)

['0.014', '0.008', '0.028', '0.014', '0.000', '0.002', '0.004', '0.000', '0.002', '0.024', '0.001', '0.000', '0.000', '0.001', '0.004', '0.005', '0.873', '0.000', '0.000', '0.000', '0.000', '0.000', '0.003', '0.001', '0.000', '0.000', '0.000', '0.000', '0.016', '0.000', '0.000']


In [17]:
print('Decision was made by "{}".'.format(X.columns[16]))

Decision was made by "trip_distance".


In [18]:
mask = list(map(lambda x: True if x > 0.01 else False, rfr.feature_importances_))

In [19]:
# Most important features
X_train.columns[mask]

Index(['pickup_longitude', 'dropoff_longitude', 'dropoff_latitude',
       'pickup_year', 'trip_distance', 'dropoff_borough_others'],
      dtype='object')